<a href="https://colab.research.google.com/github/vsnupoudel/Next-word-prediction-using-Probability-in-a-Corpus/blob/master/GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup:

1) Make sure GPU is enabled, go to edit->notebook settings->Hardware Accelerator GPU

2) Make a copy to your google drive, click on copy to drive in panel

Note: Colab will reset after 12 hours make sure to save your model checkpoints to google drive around 10-11 hours mark or before, then go to runtime->reset all runtimes. Now copy your train model back into colab and start training again from the previous checkpoint.

clone and cd into repo, nshepperd's fork https://github.com/nshepperd/gpt-2

In [1]:
!git clone https://github.com/nshepperd/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 435, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 435 (delta 20), reused 45 (delta 13), pack-reused 371
Receiving objects: 100% (435/435), 4.48 MiB | 14.15 MiB/s, done.
Resolving deltas: 100% (221/221), done.


In [2]:
cd gpt-2

/content/gpt-2


Install requirements

In [3]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 87 kB 3.4 MB/s 
     |████████████████████████████████| 601 kB 39.6 MB/s 
     |████████████████████████████████| 57 kB 4.8 MB/s 
     |████████████████████████████████| 48 kB 4.6 MB/s 
     |████████████████████████████████| 58 kB 5.7 MB/s 
  Created wheel for regex: filename=regex-2017.4.5-cp37-cp37m-linux_x86_64.whl size=534445 sha256=3949ae989d8f9bbb0ba5cd5bf233e253a9d7db3d6a32658a2b81389298a64853
  Stored in directory: /root/.cache/pip/wheels/3d/e8/a5/d4894e7ef29935f75c6074409ce8ca80a0271f0ce2a30da5d3
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=96b81e0bde766f00402b5a568f92993bd946ba51ef9828f38054ee83a87d338a
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built regex fire
  Attempting uninstall: idna
    Found existing installation: idna 2.10
    Uninstalling idna-2.10:
      Successfully uninstalled idna-2.10
  Attempting uninst

Mount drive to access google drive for saving and accessing checkpoints later. Have to log in to your google account

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Download the model data

In [5]:
!python3 download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 400kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 4.31Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 745kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:11, 41.6Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 3.32Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 3.26Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 3.05Mit/s]                                                       


In [6]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 369kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 5.06Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 767kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:35, 39.6Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 7.20Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 4.33Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 2.72Mit/s]                                                       


encoding

In [7]:
!export PYTHONIOENCODING=UTF-8

Fetch checkpoints if you have them saved in google drive

In [22]:
!cp -r /content/drive/My\ Drive/checkpoint/ /content/gpt-2/ 

In [24]:
!ls -alh /content/drive/My\ Drive/checkpoint/run1

total 0



Let's get our train on! In this case the file is A Tale of Two Cities (Charles Dickens) from Project Gutenberg. To change the dataset GPT-2 models will fine-tune on, change this URL to another .txt file, and change corresponding part of the next cell. Note that you can use small datasets if you want but you will have to be sure not to run the fine-tuning for too long or you will overfit badly. Roughly, expect interesting results within minutes to hours in the 1-10s of megabyte ballpark, and below this you may want to stop the run early as fine-tuning can be very fast.

In [25]:
# !wget https://www.gutenberg.org/files/98/98-0.txt
# !gdown --id 1hlvOXJqe3fdxjQilNfLRwPud_KWcOVEE -O '/content/en_US.zip'
# !unzip -o /content/en_US.zip 

In [12]:
%%bash
cd /content/gpt-2/en_US/
cat en_US.blogs.txt en_US.news.txt en_US.twitter.txt > train_text.txt

In [13]:
!cd /content/gpt-2/

In [37]:
with open("/content/gpt-2/en_US/en_US.twitter.txt", 'r') as f:
  smallerfile = f.readlines(99999)
  with open("/content/gpt-2/en_US/smallerfile.txt", "w") as file1:
    # Writing data to a file
    file1.writelines(smallerfile)


Start training, add --model_name '345M' to use 345 model

In [38]:
!ls -alh /content/gpt-2/en_US/

total 1.1G
drwxr-xr-x 2 root root 4.0K Jan  9 09:59 .
drwxr-xr-x 8 root root 4.0K Jan  9 08:59 ..
-rw-r--r-- 1 root root 201M Jul 22  2014 en_US.blogs.txt
-rw-r--r-- 1 root root 197M Jul 22  2014 en_US.news.txt
-rw-r--r-- 1 root root 160M Jul 22  2014 en_US.twitter.txt
-rw-r--r-- 1 root root  98K Jan  9 10:04 smallerfile.txt
-rw-r--r-- 1 root root 557M Jan  9 08:56 train_text.txt


In [41]:
!PYTHONPATH=src ./train.py --dataset /content/gpt-2/en_US/smallerfile.txt --model_name '117M'

Streaming output truncated to the last 5000 lines.
[1182 | 1293.51] loss=0.08 avg=0.09
[1183 | 1294.41] loss=0.07 avg=0.09
[1184 | 1295.32] loss=0.24 avg=0.09
[1185 | 1296.24] loss=0.08 avg=0.09
[1186 | 1297.15] loss=0.05 avg=0.09
[1187 | 1298.04] loss=0.07 avg=0.09
[1188 | 1298.94] loss=0.04 avg=0.09
[1189 | 1299.84] loss=0.06 avg=0.09
[1190 | 1300.74] loss=0.05 avg=0.09
[1191 | 1301.65] loss=0.05 avg=0.09
[1192 | 1302.54] loss=0.04 avg=0.09
[1193 | 1303.45] loss=0.08 avg=0.09
[1194 | 1304.35] loss=0.05 avg=0.08
[1195 | 1305.26] loss=0.09 avg=0.08
[1196 | 1306.16] loss=0.04 avg=0.08
[1197 | 1307.06] loss=0.07 avg=0.08
[1198 | 1307.96] loss=0.04 avg=0.08
[1199 | 1308.87] loss=0.06 avg=0.08
Generating samples...
======== SAMPLE 1 ========
 like you, and you know I want to be your wife and baby.
what a story - Happy Friday! #shoutoutoutgods
man so dedicated ive never felt so loved even if I love youI will never make it like this You must love every girl i lover you <3
so what the f–k is 

In [42]:
!ls -alh /content/gpt-2/checkpoint/run1/

total 1.9G
drwxr-xr-x 2 root root 4.0K Jan  9 11:19 .
drwxr-xr-x 3 root root 4.0K Jan  9 08:59 ..
-rw-r--r-- 1 root root  200 Jan  9 11:19 checkpoint
-rw-r--r-- 1 root root    5 Jan  9 11:19 counter
-rw-r--r-- 1 root root 266K Jan  9 11:23 events.out.tfevents.1641722773.f2d20458afe9
-rw-r--r-- 1 root root 475M Jan  9 10:24 model-1000.data-00000-of-00001
-rw-r--r-- 1 root root 5.1K Jan  9 10:24 model-1000.index
-rw-r--r-- 1 root root 2.6M Jan  9 10:24 model-1000.meta
-rw-r--r-- 1 root root 475M Jan  9 10:42 model-2000.data-00000-of-00001
-rw-r--r-- 1 root root 5.1K Jan  9 10:42 model-2000.index
-rw-r--r-- 1 root root 2.6M Jan  9 10:42 model-2000.meta
-rw-r--r-- 1 root root 475M Jan  9 11:01 model-3000.data-00000-of-00001
-rw-r--r-- 1 root root 5.1K Jan  9 11:01 model-3000.index
-rw-r--r-- 1 root root 2.6M Jan  9 11:01 model-3000.meta
-rw-r--r-- 1 root root 475M Jan  9 11:19 model-4000.data-00000-of-00001
-rw-r--r-- 1 root root 5.1K Jan  9 11:19 model-4000.index
-rw-r--r-- 1 root root 2.

Save our checkpoints to start training again later

In [43]:
!cp -r /content/gpt-2/checkpoint/ /content/drive/My\ Drive/

Load your trained model for use in sampling below (117M or 345M)

In [48]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/4000M/

In [ ]:
# !cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/

Generate conditional samples from the model given a prompt you provide -  change top-k hyperparameter if desired (default is 40),  if you're using 345M, add "--model-name 345M"

In [60]:
!python3 src/interactive_conditional_samples.py --top_k 10 --model_name "117M" --temperature=0.7 --length=2 

2022-01-09 11:54:54.917538: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
Model prompt >>> What a nice sunny
======================================== SAMPLE 1 ========================================
 day,
Model prompt >>> I love my
======================================== SAMPLE 1 ========================================
 momma
Model prompt >>> Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/ops.py", line 5570, in get_controller
    yield default
  File "src/interactive_conditional_samples.py", line 73, in interact_model
    raw_text = input("Model prompt >>> ")
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent

In [53]:
# !cp /content/gpt-2/models/345M/vocab.bpe /content/gpt-2/models/4000M

To check flag descriptions, use:

In [46]:
!python3 src/interactive_conditional_samples.py -- --help

NAME
    interactive_conditional_samples.py - Interactively run the model :model_name=124M : String, which model to use :seed=None : Integer seed for random number generators, fix seed to reproduce results :nsamples=1 : Number of samples to return total :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples. :length=None : Number of tokens in generated text, if None (default), is determined by model hyperparameters :temperature=1 : Float value controlling randomness in boltzmann distribution. Lower temperature results in less random completions. As the temperature approaches zero, the model will become deterministic and repetitive. Higher temperature results in more random completions. :top_k=0 : Integer value controlling diversity. 1 means only 1 word is considered for each step (token), resulting in deterministic completions, while 40 means 40 words are considered at each step. 0 (default) is a special setting meaning no restrictions. 40 generally is a g

Generate unconditional samples from the model,  if you're using 345M, add "--model-name 345M"

In [59]:
!python3 src/generate_unconditional_samples.py --model_name "117M" | tee /tmp/samples

2022-01-09 11:50:15.936280: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
That is not what I 
======================================== SAMPLE 1 ========================================
RT: Hey Sherry! Howdy is always online and helpful!
i do some research for me and then try to find information I can agree to
(Interrupting interferon over warning interferes with the clock in ms. High Signal Semiconductance Delay.)
Thanks! I am aware of it.
As a child my love, you were most likely extremely observant. With each passing month, the more we learn, the more we find we are, the lesser our time is.
I could go for a high drama year, but I am extremely wary of moving forward if I don't learn to love.
Lucille Inspirations is a phonemic circus

To check flag descriptions, use:

In [ ]:
!python3 src/generate_unconditional_samples.py -- --help